## Figure 1B - Comparison of RNA Content between MTCs and Total Lysate

Here we plot the transcript by transcript RPM (RReads Per Million) of all genes > 0 reads in the MTC-protected sample and the total lysate sample. 

In [1]:
import pandas as pd
import os
from bokeh.io import push_notebook, show, output_notebook, export_svg
output_notebook()
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper
import datetime
import numpy as np
from scipy.stats import gaussian_kde

Loading BokehJS ...

First we load in our data.  We will load in all the datasets associated with the experiment, though we will only be plotting/looking at the 'Ecoli_Lysate' and the 'E_coli_Cap' data here:

In [5]:
df_dir = '../../Processed Data Files/dataframe files/230117Li/'
prefix = '230117Li_Mixing_'
df_dict = dict()
names = []
# Fetch all the relevant dataframe files and put them in a useable format:
for file in os.listdir(df_dir):
    if file.endswith('_dataframe.txt') and file.startswith(prefix):
        name = file[file.find(prefix)+len(prefix):]
        name = name[:name.find('_1')]
        new_df  = pd.read_csv(''.join([df_dir,file]))
        new_df['Length'] = new_df['Stop'] - new_df['Start']
        df_dict[name] = new_df
        names.append(name)
print(f'\nHere is the list of unique samples for which there are dataframe files: \n\n{names}\n')

# a name dictionary for converting between the datafram name and a form nicer for plots:
names_dict = {'Ecoli_Cap':'E. coli Capsule', 'Mix_Cap':'Mixture Capsule', 'Ecoli_Lysate':'E. coli Lysate', 
              'Mix_Lysate':'Mixture Lysate', 'Bsub_Lysate':'B. subtilis Lysate'}


Here is the list of unique samples for which there are dataframe files: 

['Bsub_Lysate', 'Ecoli_Cap', 'Ecoli_Lysate', 'Mix_Cap', 'Mix_Lysate']



Next we have the helper function to convert tread counts into RPM:

In [6]:
def into_RPM(df_dict, name):
    '''This felper function gets the dataframe for a particular experiment and converts its reads into RPM'''
    sample_df = df_dict[name].copy()
    sample_df = sample_df.rename(columns = {'Counts':name})
    sample_df[name] = sample_df[name]/(sum(sample_df[name])/1000000) #RPM]
    
    return sample_df

Now we will take the data we need in order to make our comparison between the MTC-protected RNA and the general lysate RNA. We also include steps which affect the plot appearance. 

**Note** This notebook will output both the an interactive plot below and an svg image of the plot. In order for the svg export to work you need to have selenium and the firefox/geckodriver libraries installed via conda. Feel free to comment these parts of the code out if all you want is the interactive plot.

In [11]:
x = 'Ecoli_Cap' # MTC-protected RNA sample
y = 'Ecoli_Lysate' # General Lysate sample - the same as the MTC-protected RNA sample.

x_df = into_RPM(df_dict, x)
y_df = into_RPM(df_dict, y)
plot_df = x_df.merge(y_df)

plot_df.drop(plot_df[(plot_df['Name'].isin(['Capsule', 'lacI', 'error']))].index, inplace = True) # drop capsule reads from plot
plot_df.drop(plot_df[plot_df[x] <= 1].index, inplace = True) #drop any transcripts which don't have any reads in the MTC sample
plot_df.drop(plot_df[plot_df[y] <= 1].index, inplace = True) # drop any transcripts which don't have any reads in the total lysate

#Now we will make a density plot based on the log RPM of each dataset:
xy = np.vstack([np.log(plot_df[x]), np.log(plot_df[y])])
plot_df['Density'] = gaussian_kde(xy)(xy)
color_mapper = LinearColorMapper(
    palette='Viridis256',
    low = min(plot_df['Density']),
    high = max(plot_df['Density']),
)

p = figure(
    y_axis_type = "log", x_axis_type = "log",
    aspect_scale = 1, width = 450, height = 450,
    output_backend = "svg", tooltips = [("Gene", "@Name")]
)

#configure title appearance:
p.title = "MTC and Lysate RNA Level Agreement"
p.title.text_font = "arial"
p.title.text_font_size = "22px"
p.title.align = "center"
p.title.text_font_style = "normal"

p.xaxis.axis_label = "MTC RNA Level (RPM)"
p.yaxis.axis_label = "Lysate RNA Level (RPM)"
p.axis.ticker = [10**0, 10**2, 10**4, 10**6]
p.axis.major_label_text_font = "arial"
p.axis.major_label_text_font_size = "20px"
p.axis.axis_label_text_font_style = "normal"
p.axis.axis_label_text_font = "arial"
p.axis.axis_label_text_font_size = "22px"

p.background_fill_color = "whitesmoke"
p.grid.grid_line_color = "white"
p.grid.grid_line_width = 4

p.circle(x=x,
         y=y,
         source=plot_df,
         size=5,
         fill_alpha=0.8,
         line_alpha=0,
         color={'field': 'Density', 'transform': color_mapper},
        )
p.line(x=[10**(0), 10**4], y=[10**(0), 10**4], color = "black", width = 4)
p.line(x=[10**(1), 10**4], y=[10**(0), 10**3], color = "black", width = 4)
p.line(x=[10**(0), 10**3], y=[10**(1), 10**4], color = "black", width = 4)


show(p)
export_svg(p, filename = f'./1C_{datetime.date.today()}.svg')

['./1C_2023-07-31.svg']

In [ ]:
plot_df['Log_Dif'] = np.log10(plot_df[x]) - np.log10(plot_df[y])
plot_df['Abs_Log_Dif'] = np.abs(plot_df['Log_Dif'])